In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import collections
import category_encoders as ce # импорт для работы с кодировщиком

# импортируем библиотеки для визуализации
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Загружаем данные

In [ ]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [ ]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

In [ ]:
# Подгрузим наши данные из соревнования

DATA_DIR = '/kaggle/input/sf-booking/'
df_train = pd.read_csv(DATA_DIR+'/hotels_train.csv') # датасет для обучения
df_test = pd.read_csv(DATA_DIR+'hotels_test.csv') # датасет для предсказания
sample_submission = pd.read_csv(DATA_DIR+'/submission.csv') # самбмишн

In [ ]:
df_train.info()

In [ ]:
df_train.head(2)

In [ ]:
df_test.info()

In [ ]:
df_test.head(3)

In [ ]:
sample_submission.head(5)

In [ ]:
sample_submission.info()

In [ ]:
# ВАЖНО! дря корректной обработки признаков объединяем train и test в один датасет
df_train['train'] = 1 # помечаем где у нас train
df_test['train'] = 0 # помечаем где у нас test
df_test['reviewer_score'] = 0 # в тесте у нас нет значения reviewer_score, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

# Изучаем данные

In [ ]:
data.info()

Предлагаемый для анализа датасет состоит  из 17 признаков + 1 добавили(train).\
Данные распределены по следующим типам: 4 признака - float, 6 - int, 8 - object.\
review_date представлены типом object, необходимо будет его преобразовать в datetime.\
В признаках lat и lng есть пропуски, нужно будет их заполнить.

In [ ]:
# Пропущенные данные
print(data.isnull().sum()[data.isnull().sum() > 0]) 

По рекомендациям с вебинара, строки из датасета не удаляем, поэтому проверку на дубликаты не проводим

# Очистка данных, создание новых признаков  

## 1.Hotel_address

In [ ]:
print(data['hotel_address'][10])
print(data['hotel_address'][100])
print(data['hotel_address'][1000])
print(data['hotel_address'][10000])
print(data['hotel_address'][100000])


Из признака **hotel_address** можно извлечь информацию о стране и городе отеля. Напишем две функции для извлечения города и страны из адреса отеля.

In [ ]:
# На вход данной функции поступает строка с адресом.
def get_country(address):
# Методом split получаем список слов в строке и заносим его в переменную address_list.
    address_list = address.split(' ')
# Обрезаем список, оставляя в нём только последний элемент - страна нахождения гостиницы
    country = address_list[-1]
    # для Соединенного королевства формат адреса отличается от иных, поэтому для него отдельная обработка
    if country == 'Kingdom':
        country = address_list[-2]+' '+address_list[-1]
    return country

# На вход данной функции поступает строка с адресом.
def get_city(address):
# Методом split получаем список слов в строке и заносим его в переменную address_list.
    address_list = address.split(' ')
# Обрезаем список, оставляя в нём только предпоследний элемент - город нахождения гостиницы
    city = address_list[-2]
    # для Соединенного королевства формат адреса отличается от иных, поэтому для него отдельная обработка
    if address_list[-1] == 'Kingdom':
        city = address_list[-5]
    else:
        city = address_list[-2]
    return city

In [ ]:
# создаем два новых признака 'country_hotel' и 'city_hotel'

data['hotel_country'] = data['hotel_address'].apply(get_country)
data['hotel_city'] = data['hotel_address'].apply(get_city)
data.info()

In [ ]:
# Определяем уникальные страны и города местонахождения гостиниц
print(data['hotel_country'].unique())
print(data['hotel_city'].unique())

Количество городов и стран совпадает, каждый город соответствует конкретной стране, поэтому признак с городом можно смело удалять. Признак со страной лучше всего оставить, т.к. значения в нем частично совпадают со значениями из признака **reviewer_nationality**, это можно использовать в дальнейшем для создания нового признака.

Сам признак со страной отеля закодируем используя OneHotEncoding.

In [ ]:
encoder = ce.OneHotEncoder(cols=['hotel_country'], use_cat_names=True) # указываем столбец для кодирования
type_bin = encoder.fit_transform(data['hotel_country'])
data = pd.concat([data, type_bin], axis=1)

In [ ]:
data['hotel_country'].value_counts()

Наибольшее количество отзывов, которые оставили пользователи, относятся к отелям из Великобритании - более половины всех отзывов!

## 2. Review_date

In [ ]:
# Преобразуем в тип datetime
data['review_date'] = pd.to_datetime(data['review_date'], dayfirst=True)

#Создадим новый признак квартал в котором была выставлена оценка и написан отзыв
data['review_quarter'] = data['review_date'].dt.to_period('Q')
data['review_quarter'][:5]

In [ ]:
# Выведим в признак review_quarter только номер квартал и преобразуем в тип int
data['review_quarter'] = data['review_quarter'].apply(lambda x: int(str(x)[-1]))

In [ ]:
data['review_quarter'].value_counts()

Наибольшее количество отзывов приходиться на 2 и 3 кварталы, с апреля по сентябрь. На эти месяцы как правило приходиться сезон отпусков.

In [ ]:
#удалим столбец review_date
data.drop('review_date', axis=1, inplace=True)

## 3. lat и lnf. Заполняем пропуски

In [ ]:
# перебираем все страны, напомним что их всего 6
for country in data['hotel_country'].value_counts().index:
    mask = data['hotel_country'] == country
    # проверяем есть ли у отелей рассматриваемой страны пропуски в признаках lat и lng
    if data[mask].isnull().sum()['lat'] > 0 and data[mask].isnull().sum()['lng'] > 0:
        # если пропуски есть, то берем в качестве значения для заполнения пропуска среднее 
        # значение координат отелей данной страны
        value = {'lat': data[mask]['lat'].mean(), 'lng': data[mask]['lng'].mean()}
        # заполняем полученным значением пропуски
        data[mask] = data[mask].fillna(value)

In [ ]:
# Пропущенные данные
data.isnull().sum()[data.isnull().sum() > 0]

## 4. Negative_review и positive_review

По двум этим признакам проведем анализ тональности через "Анализатор настроений" ))


In [ ]:
data['negative_review'].value_counts()

In [ ]:
data['positive_review'].value_counts()

Самым популярным значением в признаке с негативными отзывами является "No Negative", а в признаке с положительным отзывом - "No Positive". Анализатор настроений может воспринять "No Negative" как негативный отзыв исходя из слова negative, но на самом деле ничего негативного тут нет. С "No Positive" аналогичная история. Поэтому чтобы не сбить с толку анализатор заменим:
- "No Positive" на "Negative"
- "No Negative" на "Positive"

In [ ]:
data['negative_review'] = data['negative_review'].apply(lambda x: x.replace('No Negative', 'Positive'))
data['positive_review'] = data['positive_review'].apply(lambda x: x.replace('No Positive', 'Negative'))

In [ ]:
# сделаем функции для оценки сентимента в негативных отзывах и в позитивных отзывах
def sent_analize_neg(row):
    sent_dict = sent_analyzer.polarity_scores(row['negative_review'])
    row['neg_pos_rev'] = sent_dict['pos']
    row['neg_neg_rev'] = sent_dict['neg']
    row['neg_neu_rev'] = sent_dict['neu']
    row['neg_compound_rev'] = sent_dict['compound']
    return row

def sent_analize_pos(row):
    sent_dict = sent_analyzer.polarity_scores(row['positive_review'])
    row['pos_pos_rev'] = sent_dict['pos']
    row['pos_neg_rev'] = sent_dict['neg']
    row['pos_neu_rev'] = sent_dict['neu']
    row['pos_compound_rev'] = sent_dict['compound']
    return row

In [ ]:
sent_analyzer = SentimentIntensityAnalyzer()

In [ ]:
# анализ сентимента негативных отзывов
data = data.apply(sent_analize_neg, axis=1)

In [ ]:
# анализ сентимента позитивных отзывов
data = data.apply(sent_analize_pos, axis=1)

In [ ]:
# также добавим еще один признак с разницей количества негативных и позитивных слов 
data['positive_to_negative_diff'] = data['review_total_positive_word_counts'] - data['review_total_negative_word_counts']

## 5. Days_since_review

In [ ]:
data['days_since_review'].head()

In [ ]:
# Преобразуем данный признак в тип int.
data['days_since_review'] = data['days_since_review'].apply(lambda x: int(x.split()[0]) )


## 6. Reviewer_nationality

In [ ]:
data['reviewer_nationality'][10]


In [ ]:
data['reviewer_nationality'][20]

In [ ]:
data['reviewer_nationality'][30]

В признаке **reviewer_nationality** в данных есть пробел  в начале и конце  строки. Уберем его.
Добавим в датасет признак, который показывает является ли пользователь, оставляющий отзыв, жителем той же страны, где находится отель, на который он оставляет отзыв. 

In [ ]:
data['reviewer_nationality'] = data['reviewer_nationality'].apply(lambda x: x.strip())
data['same_country'] = np.where(data['hotel_country'] == data['reviewer_nationality'], 1, 0)

In [ ]:
data['reviewer_nationality'].value_counts()

Список национальностей содержит 227 уникальных значений. Оставим 5 уникальных значений, а остальные обозначим как Other. Далее закодируем признак.

In [ ]:
nationality_top5 = list(data['reviewer_nationality'].value_counts().nlargest(5).index)

data['reviewer_nationality'] = data['reviewer_nationality'].apply(lambda x: x if x in nationality_top5 else 'Other')

encoder = ce.OneHotEncoder(cols=['reviewer_nationality'], use_cat_names=True) # указываем столбец для кодирования
type_bin = encoder.fit_transform(data['reviewer_nationality'])
data = pd.concat([data, type_bin], axis=1)

## 7. Tag 

In [ ]:
data['tags']

In [ ]:
data['tags'][0]

In [ ]:
data['tags'][358]

Получим из признака **tags** перечень наиболее употребляемых тэгов

In [ ]:
# преобразуем в список
def tags_list(tags):
    # убираем из текста квадратные скобки
    tags=tags[1:-1]
    # сплитуем по запятой с пробелом и возвращаем
    return tags.split(', ')

all_tags=data['tags'].apply(tags_list)

# теперь через counter можем посчитать повторяемость тэгов
c = collections.Counter()
for tags in all_tags:
    for tag in tags:
        c[tag] += 1

# анализируем 70 наиболее упоминаемых тэгов
c.most_common()[0:69]

**Вывод из анализа тэгов:**

на основе анализа наиболее употребляемых тэгов можем создать следующие признаки:
- количество ночей (Stayed _n_ nights)
- тип поездки (Busuness trip или Leisure trip)
- признак написания отзыва с мобильного телефона (Submitted from a mobile device)
- состав/тип путешественников (Couple, Solo traveler, Group, Family with young children, Family with older children, Travelers with friends) - сумма упоминаний таких тэгов равна размеру датасэта

Все остальные тэги характеризуют типы номеров, которых имеется достаточно большое количество, в связи с этим не будем из них создавать дополнительные признаки.

In [ ]:
# Создание нового признака с количеством ночей, проведенных в отеле

def stayed(tag):
    # находим положение слова Stayed в отзыве
    position = tag.find('Stayed')
    # если слова в отзыве нет, то position будет равен -1. Проверяем это и в таком случае возвращаем 0
    if position < 0:
        return 0
    else:
        # Иначе - берем срез из 2х символов после слова Stayed, убираем пробелы и преобразуем в целое число
        return int(tag[position+7:position+9].strip())    

data['stayed_nights'] = data['tags'].apply(stayed)

# data['stayed_nights'].value_counts()

В качестве альтернативного варианта, данный признак (кол-во ночей) можно попробовать закодировать только четырьмя значения: 1 ночь, 2 ночи, 3 ночи, и более 3х ночей, т.к. количество отзывов с 1-3 ночами составляет 82%, поэтому применив правило Паретто, предлагается остальными значениями пренебречь объединив их в одну категорию.

In [ ]:
# Создание нового признака с типом путешественников

# делаем список возможных типов путешественников
types_of_travelers = [' Couple ', ' Solo traveler ', ' Group ', ' Family with young children ', ' Family with older children ', ' Travelers with friends ']

def travaler_type (tag):
    for types in types_of_travelers:
        if types in tag:
            return types

data['travaler_type'] = data['tags'].apply(travaler_type)

Закодируем признак с типом путешественников методом OneHotEncoding

In [ ]:
encoder = ce.OneHotEncoder(cols=['travaler_type'], use_cat_names=True) # указываем столбец для кодирования
type_bin = encoder.fit_transform(data['travaler_type'])
data = pd.concat([data, type_bin], axis=1)

In [ ]:
# Создание нового признака с признаком написания отзыва с мобильного телефона
phrase = ' Submitted from a mobile device '
data['mobile_review'] = data['tags'].apply(lambda x : 1 if phrase in x else 0)

In [ ]:
# Создание нового признака с типом поездки

# делаем список возможных типов поездки
types_of_trips = [' Leisure trip ', ' Business trip ']

def trip_type (tag):
    for types in types_of_trips:
        if types in tag:
            return types

data['trip_type'] = data['tags'].apply(trip_type)

In [ ]:
# после добавления признаков проверяем пустые значения
print(data.isnull().sum()[data.isnull().sum() > 0]) 

В признаке с типом поездки имеются более 15 тыс строк с пропущенными значениями.
Можно проанализировать эти пустые значения по признаку типов путешественников и подставить вместо них модальные значения, то получим признак, содержащий 2 значения: деловая поездка, либо отпуск (наиболее вероятно что путешествия с детьми, друзьями или вдвоём - относятся к типу поездки - отпуск, а одиночные поездки наиболее вероятно - деловые). Но если закодировать данный признак через OneHotEncoding, то получим 2 признака, которые явно будут обладать высоким показателем корреляции, в результате чего 1 признак придется удалять. 
Поэтому меняем подход и делаем просто 1 признак - Leisure trip, т.к. таких поездок в датасете преобладающее большинство (более 417 тыс). А созданный нами признак **trip_type** удаляем.

In [ ]:
# Создание нового признака с признаком отнесения поездки к отпуску
phrase = ' Leisure trip '
data['leisure_trip'] = data['tags'].apply(lambda x : 1 if phrase in x else 0)

#удалим столбец trip_type
data.drop('trip_type', axis=1, inplace=True)

In [ ]:
# еще раз проверяем пустые значения
print(data.isnull().sum()[data.isnull().sum() > 0]) 

In [ ]:
plt.rcParams['figure.figsize'] = (15,10)
sns.heatmap(data.drop(['train'], axis=1).corr(), annot=True)

In [ ]:
 # модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
object_columns = [s for s in data.columns if data[s].dtypes == 'object']
data.drop(object_columns, axis = 1, inplace=True)

In [ ]:
data.info()

In [ ]:
# Теперь выделим тестовую часть
train_data = data.query('train == 1').drop(['train'], axis=1)
test_data = data.query('train == 0').drop(['train'], axis=1)

y = train_data.reviewer_score.values            # наш таргет
X = train_data.drop(['reviewer_score'], axis=1)

In [ ]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

In [ ]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [ ]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [ ]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

In [ ]:
# Получение средней абсолютной ошибки
def mean_absolute_percentage_error(y_tr, y_pr):
    y_tr, y_pr = np.array(y_tr), np.array(y_pr)
    return np.mean(np.abs((y_tr - y_pr) / y_tr)) * 100

print('MAPE:', round(mean_absolute_percentage_error(y_test, y_pred), 2))

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAPE:', metrics.mean_absolute_error(y_test, y_pred))

In [ ]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

In [ ]:
test_data.sample(10)

In [ ]:
test_data = test_data.drop(['reviewer_score'], axis=1)

In [ ]:
sample_submission

In [ ]:
predict_submission = model.predict(test_data)

In [ ]:
predict_submission

In [ ]:
list(sample_submission)

In [ ]:
sample_submission['reviewer_score'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)


**Вывод:** достигли MAPE в размере 12.6 с использованием анализа исследовательских данных (EDA). Основным признаком датасета, оказывающим влияние на модель, является текст отзыва. Значительное воздействие на модель оказывают также созданные признаки оценки тональности текста.